p.s. 感觉习题有点不知所云，跟着自己的理解跑了一遍。

### Import Necessary Packages

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Load Data

In [2]:
df = pd.read_csv('../../SP500_HistoricalData.csv')
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df = df.set_index('Date').sort_index()
df

,Close/Last,Volume,Open,High,Low
Date,,,,,
2018-02-20,2714.00,1408201,2728.25,2747.75,2705.75
2018-02-21,2698.75,1573160,2714.50,2747.75,2693.50
2018-02-22,2711.50,1577667,2698.50,2731.00,2682.00
2018-02-23,2748.75,1203636,2712.75,2750.25,2710.00
2018-02-26,2784.50,1207718,2751.50,2786.00,2742.50
...,...,...,...,...,...
2023-02-13,4147.25,1320269,4105.00,4150.75,4078.75
2023-02-14,4145.50,2323441,4149.75,4186.50,4103.75
2023-02-15,4158.25,1567727,4141.25,4161.75,4113.00


## Exercise 3.1

### Pre-process Data and Construct Dollar Bars

#### Calculate Turnover (Using Close Price and Volumn to Estimate)

In [3]:
turnover = df['Close/Last'] * df['Volume']
turnover

Date
2018-02-20    3.821858e+09
2018-02-21    4.245566e+09
2018-02-22    4.277844e+09
2018-02-23    3.308494e+09
2018-02-26    3.362891e+09
                  ...     
2023-02-13    5.475486e+09
2023-02-14    9.631825e+09
2023-02-15    6.519001e+09
2023-02-16    8.168998e+09
2023-02-17    7.935272e+09
Length: 1259, dtype: float64

In [4]:
def getBars(series, boundary=1e10):
    chosen_index = []
    acc = 0
    for i in series.index:
        acc += series[i]
        if acc > boundary:
            chosen_index.append(i)
            acc = 0
    return chosen_index

close = df['Close/Last'][getBars(turnover)]
close

Date
2018-02-22    2711.50
2018-02-27    2747.50
2018-03-01    2678.25
2018-03-05    2718.50
2018-03-08    2739.25
               ...   
2023-02-07    4175.75
2023-02-09    4091.75
2023-02-13    4147.25
2023-02-15    4158.25
2023-02-17    4087.50
Name: Close/Last, Length: 530, dtype: float64

### Define Functions

In [5]:
def getTEvents(gRaw, thresholdSeries):
    tEvents, sPos, sNeg = [], 0, 0
    diff = gRaw.diff()
    for i in diff.index[1:]:
        h = thresholdSeries[i]
        sPos, sNeg = max(0, sPos + diff.loc[i]), min(0, sNeg + diff.loc[i])
        if sNeg < -h:
            sNeg = 0
            tEvents.append(i)
        elif sPos > h:
            sPos = 0
            tEvents.append(i)
    return pd.DatetimeIndex(tEvents)

def getDailyVol(close, span0=100):
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0>0]
    df0 = pd.Series(close.index[df0-1], index=close.index[close.shape[0] - df0.shape[0]:])
    df0 = close.loc[df0.index] / close.loc[df0.values].values - 1
    df0 = df0.ewm(span=span0).std()
    return df0

### Calculate Volatility

In [6]:
vol = getDailyVol(close)
vol

Date
2018-02-27         NaN
2018-03-01    0.027211
2018-03-05    0.022751
2018-03-08    0.018779
2018-03-13    0.016577
                ...   
2023-02-07    0.020279
2023-02-09    0.020287
2023-02-13    0.020172
2023-02-15    0.019973
2023-02-17    0.019927
Name: Close/Last, Length: 529, dtype: float64

### Apply the Filter - 3.1(a)

In [7]:
returns = close.pct_change()
tEvents = getTEvents(returns, vol)
filtered_close = close[tEvents]
filtered_close

2018-03-01    2678.25
2018-03-05    2718.50
2018-03-16    2756.00
2018-03-23    2597.75
2018-03-27    2615.75
               ...   
2023-02-01    4132.25
2023-02-03    4147.75
2023-02-09    4091.75
2023-02-13    4147.25
2023-02-17    4087.50
Name: Close/Last, Length: 272, dtype: float64

### Use Snippet 3.4 - 3.1(b)

In [8]:
t1 = df.index.searchsorted(tEvents + pd.Timedelta(days=1))
t1 = t1[t1 < df.shape[0]]
t1 = pd.Series(df.index[t1], index=tEvents[:t1.shape[0]])
t1

2018-03-01   2018-03-02
2018-03-05   2018-03-06
2018-03-16   2018-03-19
2018-03-23   2018-03-26
2018-03-27   2018-03-28
                ...    
2023-01-30   2023-01-31
2023-02-01   2023-02-02
2023-02-03   2023-02-06
2023-02-09   2023-02-10
2023-02-13   2023-02-14
Name: Date, Length: 271, dtype: datetime64[ns]

### Construct Events

In [9]:
events = pd.DataFrame([t1], index=['t1']).T
events['trgt'] = vol
events

,t1,trgt
2018-03-01,2018-03-02,0.027211
2018-03-05,2018-03-06,0.022751
2018-03-16,2018-03-19,0.015211
2018-03-23,2018-03-26,0.021475
2018-03-27,2018-03-28,0.020451
...,...,...
2023-01-30,2023-01-31,0.020577
2023-02-01,2023-02-02,0.020665
2023-02-03,2023-02-06,0.020464
2023-02-09,2023-02-10,0.020287


### Using Triple-Barrier Labeling Method - 3.1.1(c)

In [10]:
def applyPtSlOnT1(close, events, ptSl):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    out = events[['t1']].copy(deep=True)
    if ptSl[0] > 0:
        pt = ptSl[0] * events['trgt']
    else:
        pt = pd.Series(index=events.index) # NaNs
    if ptSl[1] > 0:
        sl = -ptSl[1] * events['trgt']
    else:
        sl = pd.Series(index=events.index) # NaNsZ
    for loc, t1 in events['t1'].fillna(close.index[-1]).iteritems():
        df0 = df['Close/Last'][loc:t1] # path prices
        df0 = df0 / df['Close/Last'][loc] - 1 # path returns
        out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min() # earliest stop loss.
        out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min() # earliest profit taking.
    return out

ptSl = [1, 1]
applyPtSlOnT1(filtered_close, events, ptSl)

,t1,sl,pt
2018-03-01,2018-03-02,NaT,NaT
2018-03-05,2018-03-06,NaT,NaT
2018-03-16,2018-03-19,NaT,NaT
2018-03-23,2018-03-26,NaT,2018-03-26
2018-03-27,2018-03-28,NaT,NaT
...,...,...,...
2023-01-30,2023-01-31,NaT,NaT
2023-02-01,2023-02-02,NaT,NaT
2023-02-03,2023-02-06,NaT,NaT
2023-02-09,2023-02-10,NaT,NaT


### Get Bins - 3.1.1(d)

In [11]:
def getBins(events_, close):
    #1) prices aligned with events
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    #2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[events_.index] - 1
    if 'side' in events_:
        out['ret'] *= events_['side'] # meta-labeling
    out['bin'] = np.sign(out['ret'])
    if 'side' in events_:
        out.loc[out['ret'] <= 0,'bin'] = 0 # meta-labeling
    return out

getBins(events, close)

,ret,bin
2018-03-01,0.015028,1.0
2018-03-05,0.007633,1.0
2018-03-16,-0.013697,-1.0
2018-03-23,0.006929,1.0
2018-03-27,-0.015579,-1.0
...,...,...
2023-01-30,0.024737,1.0
2023-02-01,0.003751,1.0
2023-02-03,0.006751,1.0
2023-02-09,0.013564,1.0
